In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, inspect
from src.data_clean import DataClean
import src.transform as tr
from src.utils import padroniza_str, create_db

In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv",
    index_col=0)

In [3]:
configs = pd.read_excel("assets/metadado.xlsx", sheet_name="schema")
configs.head(10)

,tabela,id,nome_original,nome,tipo,aceita_nulo,aceita_negativo,padroniza_str
0,nycflights,1,date_time,data_hora,date,0,1,0
1,nycflights,2,arr_delay,atraso_chegada,float,1,1,0
2,nycflights,3,carrier,companhia,str,0,1,1
3,nycflights,4,flight,id_voo,str,0,1,1
4,nycflights,5,air_time,tempo_voo,float,0,0,0
5,nycflights,6,distance,distancia,float,1,1,0
6,nycflights,7,origin,origem,str,1,1,1
7,nycflights,8,dest,destino,str,1,1,1


### Tratamento da base de dados

In [4]:
df["date_time"] =  pd.to_datetime(df[["year", "month", "day", "hour", "minute"]],  dayfirst=True)
df.head(2)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,date_time
1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0,2013-01-01 05:17:00
2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0,2013-01-01 05:33:00


In [5]:
dc = DataClean(df, configs)

In [6]:
dc.select_cols()
dc.rename_cols()
dc.select_nnull_cols()
dc.select_nneg_cols()
dc.data_type()
df_tratada = dc.return_data()
df_tratada.head()

,data_hora,atraso_chegada,companhia,id_voo,tempo_voo,distancia,origem,destino
1,2013-01-01 05:17:00,11.0,UA,1545,227.0,1400.0,EWR,IAH
2,2013-01-01 05:33:00,20.0,UA,1714,227.0,1416.0,LGA,IAH
3,2013-01-01 05:42:00,33.0,AA,1141,160.0,1089.0,JFK,MIA
4,2013-01-01 05:44:00,-18.0,B6,725,183.0,1576.0,JFK,BQN
5,2013-01-01 05:54:00,-25.0,DL,461,116.0,762.0,LGA,ATL


In [7]:
df_tratada.dtypes

data_hora         datetime64[ns]
atraso_chegada           float64
companhia                 object
id_voo                    object
tempo_voo                float64
distancia                float64
origem                    object
destino                   object
dtype: object

In [8]:
df_tratada.shape

(327346, 8)

In [9]:
for col in list(configs[configs["padroniza_str"] == 1]["nome"]):
    df_tratada[col] = df_tratada.loc[:,col].apply(lambda x: padroniza_str(x))

In [10]:
df_tratada.head()

,data_hora,atraso_chegada,companhia,id_voo,tempo_voo,distancia,origem,destino
1,2013-01-01 05:17:00,11.0,UA,1545,227.0,1400.0,EWR,IAH
2,2013-01-01 05:33:00,20.0,UA,1714,227.0,1416.0,LGA,IAH
3,2013-01-01 05:42:00,33.0,AA,1141,160.0,1089.0,JFK,MIA
4,2013-01-01 05:44:00,-18.0,B6,725,183.0,1576.0,JFK,BQN
5,2013-01-01 05:54:00,-25.0,DL,461,116.0,762.0,LGA,ATL


### Criação de novas colunas

In [11]:
# Colocar aqui as novas colunas, importantando as funções desenvolvidas no trabalho
df_tratada['tempo_horas'] = df_tratada['tempo_voo'].apply(tr.calc_horas)
df_tratada['turno'] = df_tratada['data_hora'].apply(tr.classifica_turno)

In [12]:
df_tratada.head()

,data_hora,atraso_chegada,companhia,id_voo,tempo_voo,distancia,origem,destino,tempo_horas,turno
1,2013-01-01 05:17:00,11.0,UA,1545,227.0,1400.0,EWR,IAH,3.783333,MADRUGADA
2,2013-01-01 05:33:00,20.0,UA,1714,227.0,1416.0,LGA,IAH,3.783333,MADRUGADA
3,2013-01-01 05:42:00,33.0,AA,1141,160.0,1089.0,JFK,MIA,2.666667,MADRUGADA
4,2013-01-01 05:44:00,-18.0,B6,725,183.0,1576.0,JFK,BQN,3.050000,MADRUGADA
5,2013-01-01 05:54:00,-25.0,DL,461,116.0,762.0,LGA,ATL,1.933333,MADRUGADA


In [13]:
df_tratada.dtypes

data_hora         datetime64[ns]
atraso_chegada           float64
companhia                 object
id_voo                    object
tempo_voo                float64
distancia                float64
origem                    object
destino                   object
tempo_horas              float64
turno                     object
dtype: object

### Insere no banco de dados

In [14]:
con = sqlite3.connect('projeto_python.db')
cur = con.cursor()
engine = create_engine("sqlite:///projeto_python.db")

In [15]:
if not inspect(engine).has_table("flights"):
    create_db(engine)

In [18]:
from sqlalchemy import text

# Apagar a tabela corretamente
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS flights;"))
    conn.commit()


In [19]:
with engine.connect() as connection:
        df_tratada.to_sql(name='flights', con=connection, index=False, if_exists='append')

In [20]:
cur.execute('SELECT * from flights limit 10')
result = cur.fetchall()
print(result)

[('2013-01-01 05:17:00.000000', 11.0, 'UA', '1545', 227.0, 1400.0, 'EWR', 'IAH', 3.783333333333333, 'MADRUGADA'), ('2013-01-01 05:33:00.000000', 20.0, 'UA', '1714', 227.0, 1416.0, 'LGA', 'IAH', 3.783333333333333, 'MADRUGADA'), ('2013-01-01 05:42:00.000000', 33.0, 'AA', '1141', 160.0, 1089.0, 'JFK', 'MIA', 2.6666666666666665, 'MADRUGADA'), ('2013-01-01 05:44:00.000000', -18.0, 'B6', '725', 183.0, 1576.0, 'JFK', 'BQN', 3.05, 'MADRUGADA'), ('2013-01-01 05:54:00.000000', -25.0, 'DL', '461', 116.0, 762.0, 'LGA', 'ATL', 1.9333333333333333, 'MADRUGADA'), ('2013-01-01 05:54:00.000000', 12.0, 'UA', '1696', 150.0, 719.0, 'EWR', 'ORD', 2.5, 'MADRUGADA'), ('2013-01-01 05:55:00.000000', 19.0, 'B6', '507', 158.0, 1065.0, 'EWR', 'FLL', 2.6333333333333333, 'MADRUGADA'), ('2013-01-01 05:57:00.000000', -14.0, 'EV', '5708', 53.0, 229.0, 'LGA', 'IAD', 0.8833333333333333, 'MADRUGADA'), ('2013-01-01 05:57:00.000000', -8.0, 'B6', '79', 140.0, 944.0, 'JFK', 'MCO', 2.3333333333333335, 'MADRUGADA'), ('2013-01-0